## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [103]:
import pandas as pd
import numpy as np
train_X = pd.read_csv('./Day_048/train.csv', header=None)
train_Y = pd.read_csv('./Day_048/trainLabels.csv', header=None)
test_X = pd.read_csv('./Day_048/test.csv/test.csv', header=None)

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [48]:
from sklearn import metrics
clf.fit(train_X, train_Y)
y_pred = clf.predict(train_X)
y_pred
print(metrics.mean_squared_error(train_Y, y_pred))

C:\Users\jud40\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.042962625733548684


In [49]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(train_X, train_Y)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    9.6s finished
C:\Users\jud40\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(train_X, train_Y)

# 預測測試集
y_pred = clf_bestparam.predict(train_X)

C:\Users\jud40\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
print(metrics.mean_squared_error(train_Y, y_pred))

0.001195121835590184


In [118]:
y_pred_test = clf_bestparam.predict(test_X)

In [163]:
final = pd.DataFrame([int(i) for i in pd.Series(y_pred_test > 0.5).replace(True, 1)], index = np.arange(1, y_pred_test.shape[0]+1))

In [165]:
final.to_csv('./Day_048/result.csv', header = {'ID', 'Solution'})